In [ ]:
import polars as pl
import numpy as np
from numpy.random import PCG64, PCG64DXSM, Philox, MT19937, SFC64
from scipy.spatial.transform import Rotation
import os
from datetime import datetime
import logging
import glob
from novel_toy_tools.utils import generate_renditions

RANDOM_GENERATORS = {
            "PCG64": PCG64,
            "PCG64DXSM": PCG64DXSM,
            "MT19937": MT19937,
            "PHILOX": Philox,
            "SFC64": SFC64}

class NullGenerator():
    def __init__(self, out_dir:os.PathLike, object_directory:os.PathLike, rng_method, RANDOM_SEED):
        #set up the directories and metadata
        os.makedirs(out_dir, exist_ok=True)
        self.rendition_dir = os.path.join(out_dir, "renditions")
        os.makedirs(self.rendition_dir, exist_ok=True)

        #create logger
        log_path = os.path.join(out_dir, "runtime.log")
        logging.basicConfig(
            filename=log_path,
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        self.log = logging.getLogger() #TODO log


        self.meta = {
            "out_dir": out_dir,
            "log_path": log_path,
            "object_dir": object_directory,
            "rendition_dir": self.rendition_dir,
            "generation_start_time": datetime.now(), #convert this to whatever format when saving
            "euler_order": "YZX",
            "random_seed": RANDOM_SEED,
            "rng_method": rng_method,

        }

        #TODO maybe remove these
        self.object_directory = object_directory
        self.rng_method = rng_method
        self.RANDOM_SEED = RANDOM_SEED
        self.rotation_set = None

    def gen_random_rotations(self, n):
        object_files = glob.glob(os.path.join(self.object_directory, "*.obj"))
        objects = [os.path.splitext(os.path.basename(file))[0] for file in object_files]
        object_assignments = [objects[i % len(objects)] for i in range(n)]
        primary_keys = [f"{n}_{object_name}" for n, object_name in enumerate(object_assignments)]

        self.log.info("generating random rotations")
        random_generator = np.random.Generator(RANDOM_GENERATORS[self.rng_method](seed=self.RANDOM_SEED))
        rotation_set = Rotation.random(n, random_state=random_generator).as_euler(seq=self.meta["euler_order"], degrees=True)
        rng_data = [self.rng_method] * n
        self.dataframe = pl.DataFrame({
            "KID_IDFrameKey": primary_keys,
            "Euler_X": rotation_set[:, 2],
            "Euler_Y": rotation_set[:, 0],
            "Euler_Z": rotation_set[:, 1],
            "Object": object_assignments,
            "RNG_method": rng_data
        })
        self.meta.update({"rotation_data_path": os.path.join(self.meta["out_dir"], "rotation_data.csv")})

        self.dataframe.write_csv(self.meta["rotation_data_path"])
        generate_renditions(self.meta['rotation_data_path'], self.meta['rendition_dir'], self.meta['object_dir'])

In [11]:
#init some different random number generators so we can try them all
RANDOM_SEED = 42

random_generators = [ "PCG64", "PCG64DXSM", "MT19937", "PHILOX", "SFC64"]
SAMPLE_SIZE = 1000

out_directory = "/workspace/data/null_datasets"
os.makedirs(out_directory, exist_ok=True)

for random_generator in random_generators:
    out_path = os.path.join(out_directory, random_generator)
    gen = NullGenerator(out_path, "/workspace/data/PrefViews Null 3D objects", random_generator, RANDOM_SEED)
    gen.gen_random_rotations(SAMPLE_SIZE)

Saving renditions to /workspace/data/null_datasets/PCG64/renditions


Generating Renditions: 100%|██████████| 1000/1000 [00:03<00:00, 290.66it/s]


Saving renditions to /workspace/data/null_datasets/PCG64DXSM/renditions


Generating Renditions: 100%|██████████| 1000/1000 [00:03<00:00, 311.75it/s]

Saving renditions to /workspace/data/null_datasets/MT19937/renditions



Generating Renditions: 100%|██████████| 1000/1000 [00:03<00:00, 304.45it/s]


Saving renditions to /workspace/data/null_datasets/PHILOX/renditions


Generating Renditions: 100%|██████████| 1000/1000 [00:03<00:00, 282.73it/s]


Saving renditions to /workspace/data/null_datasets/SFC64/renditions


Generating Renditions: 100%|██████████| 1000/1000 [00:04<00:00, 240.70it/s]
